# Data Cleaning in SQL

In [7]:
import psycopg2 as pg
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
conn = pg.connect(
    host='localhost',
    user = 'postgres',
    password = 'password',
    database = 'sqlda4'
)

cur = conn.cursor()

# Table schema

In [9]:
schema = pd.read_sql("""select table_name, column_name, data_type from information_schema.columns where table_name = 'parking_violation' """, conn)
schema

,table_name,column_name,data_type
0,parking_violation,double_parking_violation,double precision
1,parking_violation,violation_code,bigint
2,parking_violation,street_code1,bigint
3,parking_violation,street_code2,bigint
4,parking_violation,street_code3,bigint
5,parking_violation,vehicle_expiration_date,bigint
6,parking_violation,violation_location,double precision
7,parking_violation,violation_precinct,bigint
8,parking_violation,issuer_precinct,bigint
9,parking_violation,issuer_code,bigint


# Applying functions for string cleaning
A service to provide parking violation recipients with a hard copy of the violation is being re-designed. For proper formatting of the output of the information on the report, some fields needs to be changed from the database representation. The changes are as follows:

* For proper text alignment on the form, `violation_location` values must be 4 characters in length.
* All `P-U` (pick-up truck) values in the `vehicle_body_type` column should use a general `TRK` value.
* Only the first letter in each word in the `street_name` column should be capitalized.

The `LPAD()`, `REPLACE()`, and `INITCAP()` functions will be used to effect these changes.

In [25]:
q = """
select 
    violation_location,
    lpad(violation_location::text, 4, '0'),
    vehicle_body_type,
    replace(vehicle_body_type, 'P-U', 'TRK'),
    street_name,
    initcap(street_name)

from parking_violation;
"""
pd.read_sql(q, conn)

,violation_location,lpad,vehicle_body_type,replace,street_name,initcap
0,26.0,0026,SDN,SDN,OLD BROADWAY,Old Broadway
1,26.0,0026,SDN,SDN,W 126 STREET,W 126 Street
2,26.0,0026,SUBN,SUBN,W 125 ST,W 125 St
3,26.0,0026,SDN,SDN,W 125 ST,W 125 St
4,26.0,0026,None,None,ST NICHOLAS AVENUE,St Nicholas Avenue
...,...,...,...,...,...,...
4995,81.0,0081,SDN,SDN,N/W MONROE ST,N/W Monroe St
4996,101.0,0101,SUBN,SUBN,MOTT AVENUE,Mott Avenue
4997,1.0,0001,SUBN,SUBN,S/E/C/O SEAGIRT,S/E/C/O Seagirt
4998,101.0,0101,SUBN,SUBN,SEAGIRT AVE,Seagirt Ave


# Pattern matching
There have been some concerns raised that parking violations are not being issued uniformly throughout the day. You have been tasked with associating parking violations with the time of day of issuance. You determine that the simplest approach to completing this task is to create a new column named `morning`. This field will be populated with (the integer) `1` if the violation was issued in the morning (between 12:00 AM and 11:59 AM), and, (the integer) 0, otherwise. The time of issuance is recorded in the `violation_time` column of the `parking_violation` table. This column consists of 4 digits followed by an A (for AM) or P (for PM).

In [27]:
q = """
select 
    violation_time,
    case when 
        violation_time similar to '\d\d\d\dA'
        then 1
        else 0
        end as morning

from parking_violation;
"""
pd.read_sql(q, conn)

,violation_time,morning
0,1000A,1
1,1011A,1
2,0107A,1
3,0300A,1
4,0653A,1
...,...,...
4995,0025A,1
4996,1220P,0
4997,0622P,0
4998,0628P,0


# Masking identifying information with regular expressions
To protect parking violation recipients' privacy in a new web report, all letters in the `plate_id` column must be replaced with a dash (`-`) to mask the true license plate number.

In [32]:
q = """
select 
    plate_id,
    regexp_replace(plate_id, '[A-Z]', '-', 'g'),
    regexp_replace(plate_id, '[\d]', '-', 'g')

from parking_violation;
"""
pd.read_sql(q, conn)

,plate_id,regexp_replace,regexp_replace
0,JET2661,---2661,JET----
1,JCV6523,---6523,JCV----
2,GMK6954,---6954,GMK----
3,JGX1641,---1641,JGX----
4,GDM8069,---8069,GDM----
...,...,...,...
4995,JDX1768,---1768,JDX----
4996,GVD4312,---4312,GVD----
4997,HVV5954,---5954,HVV----
4998,GCZ9584,---9584,GCZ----


# Match similar strings
From the sample of records in the `parking_violation` table, it is clear that the `vehicle_color` values are not consistent. For example, `'GRY'`, `'GRAY'`, and `'GREY'` are all used to describe a gray vehicle. In order to consistently represent this color, it is beneficial to use a single value. The primary colors `GRAY`, `BLACK`, `RED`, `BLUE`, and `YELLOW` will be used for extending the color name standardization approach.  

* `SOUNDEX() in PostgreSQL` -> 4 character code
* Available through fuzzystrmatch module

**DIFFERENCE() function** outputs the number of positions in which strings Soundex codes match

In [96]:
q = """
select 
    summons_number,
    vehicle_color,
    case 
        when difference(vehicle_color, 'gray') = 4 then 'GRAY'
        when difference(vehicle_color, 'red') = 4 then 'RED'
        when difference(vehicle_color, 'blue') = 4 then 'BLUE'
        when difference(vehicle_color, 'black') = 4 then 'BLACK'
        when difference(vehicle_color, 'yellow') = 4 then 'YELLOW'

        else vehicle_color
        end as updated_color

from parking_violation;


"""
pd.read_sql(q, conn)

,summons_number,vehicle_color,updated_color
0,1447152396,GRY,GRAY
1,1447152402,GY,GY
2,1447152554,BLK,BLACK
3,1447152580,BLK,BLACK
4,1447152724,None,None
...,...,...,...
4995,1449842616,BLK,BLACK
4996,1452218158,BLACK,BLACK
4997,1452218195,BW,BW
4998,1452218201,YELL,YELLOW


# Fill in Missing Values

You propose changing all `NULL`-valued `vehicle_body_type` records in the `parking_violations` table to `SDN`. Discussions with your team result in a decision to use a value other than SDN as a fill-in value. The body type can be determined by looking up the vehicle using its license plate number. A license plate number is present in most `parking_violation` records. Rather than using the most frequent value to replace `NULL` `vehicle_body_type` values, a placeholder value of `Unknown` will be used. 

* Replace `NULL` `vehicle_body_type` values with the string `Unknown`.

In [101]:
q = """
select 
    vehicle_body_type,
    coalesce(vehicle_body_type, 'Unknown')
from parking_violation;

"""
pd.read_sql(q, conn)

,vehicle_body_type,coalesce
0,SDN,SDN
1,SDN,SDN
2,SUBN,SUBN
3,SDN,SDN
4,None,Unknown
...,...,...
4995,SDN,SDN
4996,SUBN,SUBN
4997,SUBN,SUBN
4998,SUBN,SUBN


# Find dupliocates
* Using `ROW_NUMBER()`, you will find `parking_violation` records that contain the same `plate_id`, `issue_date`, `violation_time`, `house_number`, and `street_name`, indicating that multiple tickets were issued for the same violation.

In [112]:
q = """
select * 
from(
    select 
        summons_number,
        row_number() over(partition by
            plate_id,
            issue_date,
            violation_time,
            house_number,
            street_name
        ) - 1 as duplicate,
        plate_id,
        issue_date,
        violation_time,
        house_number,
        street_name

    from parking_violation) as sub

where duplicate > 0
order by duplicate desc
"""

pd.read_sql(q, conn)

,summons_number,duplicate,plate_id,issue_date,violation_time,house_number,street_name
0,1439569216,2,HJM6483,06/23/2019,1015P,159,EINSTEIN LOOP N
1,1410920458,1,GEX3870,06/20/2019,1030P,1520,GRAND CONCOURSE
2,1446413147,1,GFD4777,06/30/2019,1214P,3543,WAYNE AVE
3,1448947790,1,GKX9331,06/29/2019,1030P,None,S/W C/O W 45 ST
4,1452062286,1,GR8C1VIC,06/14/2019,0315P,None,RIVERBANK STATE PARK
5,1449470622,1,GUC5106,07/03/2019,1035P,1060,BEACH AVE
6,1451262127,1,GWC4311,06/30/2019,0728P,None,SURF AVE
7,1452186870,1,GXL4110,06/30/2019,0548P,170 01,118 RD
8,1449142590,1,HAT3306,06/26/2019,0938A,811,E 219 ST
9,1454273859,1,HDC7519,07/02/2019,0447A,811,HICKS ST


# Detecting invalid values with regular expressions
* Return records with a `registration_state` that does not match two consecutive uppercase letters.

In [115]:
q = """
select 
    registration_state
from parking_violation
where registration_state not similar to '[A-Z]{2}'
"""

pd.read_sql(q, conn)

,registration_state
0,99
1,99
2,99
3,99
4,99
...,...
82,99
83,99
84,99
85,99


* Return records with a `vehicle_make` not including an uppercase letter, a forward slash (`/`), or a space (`\s`).

In [118]:
q = """
select vehicle_make

from parking_violation
where vehicle_make not similar to '[A-Z/\s]+';
"""

pd.read_sql(q,conn)

,vehicle_make
0,1


# Cleaning invalid dates
* Convert the `date_first_observed` value of records with a `'0'` `date_first_observed` value into `NULL` values using the `NULLIF()` function, so that the field can be represented as a proper date.

In [121]:
q = """
select 
    date_first_observed,
    nullif(date_first_observed, '0')

from parking_violation
"""

pd.read_sql(q,conn)

,date_first_observed,nullif
0,0,NaN
1,0,NaN
2,0,NaN
3,0,NaN
4,0,NaN
...,...,...
4995,0,NaN
4996,0,NaN
4997,0,NaN
4998,0,NaN


# Converting and displaying dates

In [142]:
q = """
select 
    issue_date,
    to_date(issue_date, 'MM/DD/YYYY')
    
from parking_violation
"""

pd.read_sql(q,conn)

,issue_date,to_date
0,06/28/2019,2019-06-28
1,06/28/2019,2019-06-28
2,06/16/2019,2019-06-16
3,06/24/2019,2019-06-24
4,07/06/2019,2019-07-06
...,...,...
4995,06/12/2019,2019-06-12
4996,07/02/2019,2019-07-02
4997,06/30/2019,2019-06-30
4998,06/30/2019,2019-06-30


In [152]:
conn.close()